In [1]:
import pdfplumber
import re
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import fnmatch
import logging
import time
import json
import numpy as np
import os
import joblib
import tensorflow as tf

import lib_plot
import xmltodict
import base64

from timeit import default_timer as timer
import magic
import yaml
import logging
import logging.config


from lib_extraction import create_fus, from_xml_to_mrv_format, from_pdf_to_mrv_format, plumber_df


import xmltodict

In [69]:
data = pd.read_excel('/home/robin/Documents/DGS/data/serge_csv/mrv_format.xlsx')
result_topic = pd.read_json('/home/robin/Documents/DGS/data/serge_csv/topic_inference.json')
result_cluster = pd.read_json('/home/robin/Documents/DGS/data/serge_csv/cluster_inference.json')

In [70]:
result_cluster['label'] = result_cluster.apply(np.argmax,axis=0)
result_cluster['weight'] = result_cluster.apply(np.max,axis=0)

In [72]:
result_cluster['weight'].describe()

count    25.000000
mean      0.003214
std       0.008382
min       0.000024
25%       0.000136
50%       0.000367
75%       0.001461
max       0.038072
Name: weight, dtype: float64

In [22]:
len(result_cluster[result_cluster['weight']>0.002])

5

In [35]:
data = pd.read_pickle('/home/robin/Documents/DGS/Annexe/L3/cleaned_data_5.pkl')
mrv = pd.read_csv('/home/robin/Documents/DGS/data/declaration_mrv_complet.csv')

In [67]:
cluster = pd.read_csv('/home/robin/Documents/DGS/Annexe/L3/25_11_2020_test/cluster/mrveille_with_cluster.csv')
print(len(cluster))

76954


In [68]:
cluster['cluster_weight'].describe()

count    76954.000000
mean         0.780647
std          0.231314
min          0.001552
25%          0.602178
50%          0.866353
75%          0.994039
max          0.999949
Name: cluster_weight, dtype: float64

In [54]:
def find_deces(L):
    deces_list = ['mort','morts','meurt','mourir','mortel','deces','deceder','decede','decedee','decedes','decedait']
    deces = 0
    for elt in L :
        if elt in deces_list :
            deces = 1
    
    return deces

data['deces'] = data['text_lem'].map(find_deces)
print('déces:',len(data[data['deces']==1])/len(data)*100)
print('critique:',len(mrv[mrv['GRAVITE']=='CRITI'])/len(mrv)*100)

déces: 1.5372820126309223
critique: 3.090157756581854


In [58]:
from gensim.utils import deaccent
mrv['text'] = mrv['ETAT_PATIENT']+ ' '+ mrv['DESCRIPTION_INCIDENT']
mrv['deces'] = mrv['text'].fillna('').map(lambda x : deaccent(x.lower()).split(' ')).map(find_deces)

print('déces:',len(mrv[mrv['deces']==1])/len(mrv)*100)
print('critique:',len(mrv[mrv['GRAVITE']=='CRITI'])/len(mrv)*100)

déces: 1.1461392520206877
critique: 3.090157756581854


In [39]:
for elt in data[data['deces']==1]['text_lem'].iloc[:20] :
    print(elt)

['patient', 'intube', 'ventiler', 'convulsion', 'sevrage', 'etat', 'mal', 'intuber', 'patient', 'box', 'check', 'salle', 'faire', 'infirmier', 'reanimation', 'renfort', 'respirateur', 'operationnel', 'bout', 'ventilation', 'respirateur', 'ventiler', 'patient', 'mettre', 'defaut', 'spontanement', 'emission', 'alarme', 'sonore', 'medecin', 'ventiler', 'patient', 'manuellement', 'temps', 'prendre', 'appareil', 'salle', 'consequence', 'pouvoir', 'mettre', 'danger', 'immediat', 'deces', 'patient', 'curarise', 'pouvoir', 'mettre', 'danger', 'immediat', 'deces', 'patient', 'curarise']
['monsieur', 'monsieur', 'deceder', 'novembre', 'monsieur', 'suite', 'dysfonctionnement', 'concentrateur', 'entrainer', 'manque', 'oxygene', 'technicien', 'constater', 'probleme', 'reprise', 'appareil', 'ecarter', 'doute', 'appareil', 'envoyer', 'expertise', 'fabricant', 'deces', 'patient', 'deces', 'patient']
['monsieur', 'monsieur', 'deceder', 'novembre', 'monsieur', 'suite', 'dysfonctionnement', 'concentrateu

In [40]:
for elt in mrv[mrv['GRAVITE']=='CRITI']['DESCRIPTION_INCIDENT'].iloc[:20] :
    print(elt)




Décès de la patiente.



Décès de la patiente.







Patient revu le 20-12-2016









Patient revu le 15-12-16.









retesté avec le lot INSTI1512BA033 = négatif







Problème déjà rencontré 2 fois avec ce type de masque: nouveau-né 2000g masque taille XL, nouveau né 1000g masque taille L.











Est décédée à l'hôpital après trois semaines d'hospitalisation.



Décès par embolie gazeuse



Décès par embolie gazeuse








de la rédaction de cette déclaration.






Conférence téléphonique avec le CCLIN et l'ARLIN la semaine prochaine









Les prothèses mammaires ont été explantées avec capsulectomie totale.









Les prothèses mammaires ont été explantées avec capsulectomie totale.









Les prothèses mammaires ont été explantées avec capsulectomie totale.









Les prothèses mammaires ont été explantées avec capsulectomie totale.









Les prothèses mammaires ont été explantées avec capsulectomie totale.









Les prothèses mammaires ont été expla

In [4]:
PATH = "/home/robin/Downloads/CDA_20200511155057576_MATERIOVIGILANCE.xml"
with open(PATH) as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [15]:
res = pd.DataFrame()
E = []
for elt in data_dict['ClinicalDocument']['component']['structuredBody']['component']:
    for el in elt['section']['text']['table']['tbody']['tr']:
        if type(el)!=str :
            for i in range(len(el['td'])):
                try:
                    res[el['td'][2*i]['#text']] = [el['td'][2*i+1]['#text']]
                except:
                    try:
                        E.append(el['td'][i]['#text'])
                    except:
                        E.append('erreur !')

In [16]:
res

,Nom (3 premières lettres),Sexe,Type de DM,Libellé du produit,Référence commerciale,Fabricant : Nom,Type d'utilisateur du DM,Date de survenue,Type de lieu de survenue,Nom de l'etablissement,...,Service,Personne à contacter dans l'établissement,Téléphone du contact,Description de l'incident,Nombre de patients ou personnes concernées,Nombre de dispositifs concernés,Ce type d’incident est-il prévu dans la notice d’utilisation du fabricant ?,Classification de l'incident,Conséquences cliniques et état actuel du patient ou personne impliquée,Actions entreprises pour le patient vis-à-vis du DM
0,test,F,Masques FFP2,Masque FFP2 Op Air Pro,M52210-WH,KOLMI,Professionnels de santé,30/04/2020,Etablissement de santé,Centre Hospitalier d'Alençon-Mamers,...,UHCD,XXXXX XXXXX,0233323020,les élastiques du masque FFP2 cassent lors de ...,1,1,NSP,Menace importante pour la santé publique,Implique l'exposition directe du soignant à un...,Consommation d'un nouveau masque FFP2


In [18]:
import re

filename = PATH.split('/')[-1].lower()
if ('matériovigilance' in filename) or ('materiovigilance' in filename):
    TYPE = 'Matériovigilance'
elif ('réactovigilance' in filename) or ('réactovigilance' in filename):
    TYPE = 'Réactovigilance'
else :           
    TYPE = ''

number = re.findall(r'\d+', filename)
if len(number)>0:
    NUMERO = number[0]
else : 
    NUMERO = ''

In [20]:
NUMERO

'20200511155057576'